In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pickle

In [4]:
df = pd.read_csv('insurance.csv')
df.head(2)


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523


In [5]:
# Divide into X and Y variables

X = df.drop('charges',axis = 1)
Y = df[['charges']]

In [6]:
# Train and Test Split

X_train_full, X_test, Y_train_full, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_val, Y_test, Y_val = train_test_split(X_train_full, Y_train_full, test_size=0.2, random_state = 42)


In [7]:
# Feature Engg

label_sex = LabelEncoder()
label_smoke = LabelEncoder()
onehot_region = OneHotEncoder(sparse_output=False)

X_train['sex'] = label_sex.fit_transform(X_train['sex'])
X_val['sex'] = label_sex.transform(X_val['sex'])

X_train['smoker'] = label_smoke.fit_transform(X_train['smoker'])
X_val['smoker'] = label_smoke.transform(X_val['smoker'])


X_train_trans = onehot_region.fit_transform(X_train[['region']])
X_val_trans = onehot_region.transform(X_val[['region']])
col_names = onehot_region.get_feature_names_out()
encoded_df_test = pd.DataFrame(X_train_trans,columns = col_names )
encoded_df_val = pd.DataFrame(X_val_trans,columns = col_names )



In [8]:
X_train = pd.concat([X_train.drop('region',axis = 1),encoded_df_test], axis = 1)
X_val = pd.concat([X_val.drop('region',axis = 1),encoded_df_val], axis = 1)

In [9]:
# Scaling

scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_val = scalar.transform(X_val)

In [10]:
# Save in Pickle file

with open('label_encoder_sex.pkl','wb') as file:
    pickle.dump(label_sex,file)

with open('label_encoder_smoke.pkl','wb') as file:
    pickle.dump(label_smoke,file)    

with open('onehot_region.pkl','wb') as file:
    pickle.dump(onehot_region,file)  

with open('scalar.pkl','wb') as file:
    pickle.dump(scalar,file)         

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [12]:
# Building ANN model

# model = Sequential([
#     Dense(64, activation = 'relu',input_shape = (X_train.shape[1],)),
#     Dense(32, activation = 'relu'),
#     Dense(1, activation = 'sigmoid')])